# 金融 证券 图谱搭建

根据前面数据梳理结果，下面按照节点和关系分别进行数据抽取

实体(节点)方面
1. 省份
2. 城市
3. 公司(上市公司 \ 基金管理公司 \ 基金托管公司)
4. 人（上市公司管理层）
5. 基金
6. 行业

关系方面
1. 城市 - [IN_PROVINCE] - > 省份 
2. 公司(上市公司\基金管理公司\ 基金托管公司) - [IN_CITY] - > 城市
4. 公司(上市公司) - [HAS_MANGER] - > 人（上市公司管理层）  [6类]
5. 公司(上市公司) - [IN_INDUSTRY] - > 行业
6. 基金  - [HAS_MANAGEMNET] - > 公司(基金管理公司)
6. 基金  - [HAS_CUSTODIAN] - > 公司(基金托管公司)
7. 基金  - [IN_PORTFOLIO] ->  公司(上市公司)

## 数据处理

In [1]:
import pandas as pd

In [2]:
def generate_input_files(data,dicts,types,output_path='../data/output/'):
    cols =list(dicts.keys())
    temp = data[cols]
    temp.drop_duplicates(cols,inplace=True)

    print('save header {}..')
    columns = ','.join(dicts.values())
    with open(output_path+'{}-header.csv'.format(types),'w') as f:
        f.write(columns)
    
    print('save_data ...\n',output_path+'{}.csv'.format(types))
    temp.to_csv(output_path+'{}.csv'.format(types),index=False,header=False)
    print(temp.shape)
    return temp


In [3]:
output_path = '../data/output/'

In [4]:
data_path = '../data/'

In [5]:
DATE = ''

In [6]:
company_lsit_name = f'company_lsit_all{DATE}.csv'
company_detail_name =  f'company_detail_all{DATE}.csv'
stk_managers_name = f'stk_managers{DATE}.csv'
fund_basic_name = f'fund_basic{DATE}.csv'
fund_company_name = f'fund_company{DATE}.csv'
fund_portfolio_name = f'fund_portfolio{DATE}.csv'

In [8]:
all_dicts ={}

## 1.公司
合并 处理 基金管理人\ 基金托管人\ 上市公司\

### 1.1 上市公司

In [9]:
usecols = ['ts_code','symbol','name','industry','fullname','market','exchange','list_status',
           'list_date','delist_date']

df_company_lsit_all =  pd.read_csv(data_path + company_lsit_name,dtype={"symbol":str},usecols=usecols)

In [10]:
df_company_lsit_all['list_date'] = df_company_lsit_all['list_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_company_lsit_all['delist_date'] = df_company_lsit_all['delist_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )

df_company_lsit_all['label_listed_company']='LISTED_COMPANY'

In [11]:
df_company_lsit_all['symbol'] = df_company_lsit_all['ts_code'].apply(lambda x: x.split('.')[0])

In [12]:
df_company_lsit_all.head()

,ts_code,symbol,name,industry,fullname,market,exchange,list_status,list_date,delist_date,label_listed_company
0,000001.SZ,000001,平安银行,银行,平安银行股份有限公司,主板,SZSE,L,19910403,-1,LISTED_COMPANY
1,000002.SZ,000002,万科A,全国地产,万科企业股份有限公司,主板,SZSE,L,19910129,-1,LISTED_COMPANY
2,000004.SZ,000004,国农科技,生物制药,深圳中国农大科技股份有限公司,主板,SZSE,L,19910114,-1,LISTED_COMPANY
3,000005.SZ,000005,世纪星源,环境保护,深圳世纪星源股份有限公司,主板,SZSE,L,19901210,-1,LISTED_COMPANY
4,000006.SZ,000006,深振业A,区域地产,深圳市振业(集团)股份有限公司,主板,SZSE,L,19920427,-1,LISTED_COMPANY


### 1.2  基金管理公司

#### 1.2.1 fund_company  中 基金管理人

In [13]:
# 基金管理人数据
usecols = ['name','shortname','province','city','address','office','website',
            'setup_date','end_date','main_business','org_code','credit_code']
df_fund_company = pd.read_csv(data_path + fund_company_name,usecols=usecols)

In [14]:
df_fund_company['setup_date'] = df_fund_company['setup_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_company['end_date'] = df_fund_company['end_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )

In [15]:
df_fund_company = df_fund_company.rename(columns ={'name':'fullname','shortname':'name'}).drop_duplicates()

fund_company 和 fund_basic 两张数据表中，有下面五个银行、一个券商、一个资管 名称公司简称**没有对齐**，

这里我们手工对其下：
处理 几个 ['中国民生银行', '中国光大银行', '中国工商银行', '中国建设银行', '中国农业银行'] 带有 "中国" 的公司

In [17]:
comp_simplify_dict = {
    '中国民生银行':'民生银行',
    '中国光大银行':'光大银行', 
    '中国工商银行':'工商银行', 
    '中国建设银行':'建设银行',
    '中国农业银行':'农业银行',
    '中银国际证券':'中银证券',
    '财通证券资管':'财通资管'
}

In [18]:
df_fund_company['name'] = df_fund_company['name'].apply(lambda x : comp_simplify_dict[x] if x in comp_simplify_dict.keys() else x )

In [19]:
df_fund_company.head().T

,0,1,2,3,4
fullname,北京广能投资基金管理有限公司,宏源证券股份有限公司,国元证券股份有限公司,广发证券股份有限公司,长江证券股份有限公司
name,广能基金,宏源证券,国元证券,广发证券,长江证券
province,北京,新疆,安徽,广东,湖北
city,北京市,乌鲁木齐市,合肥市,广州市,武汉市
address,北京市朝阳区北四环中路27号院5号楼2712-2715A,新疆维吾尔自治区乌鲁木齐市文艺路233号宏源大厦,安徽省合肥市梅山路18号,广东省广州市黄埔区中新广州知识城腾飞一街2号618室,湖北省武汉市江汉区新华路特8号
office,北京市朝阳区北四环中路27号院5号楼2712-2715A,新疆维吾尔自治区乌鲁木齐市文艺路233号宏源大厦,安徽省合肥市梅山路18号,"广东省广州市天河区天河北路183-187号大都会广场40楼5楼,7楼,8楼,18楼,19楼,...",湖北省武汉市江汉区新华路特8号
website,www.gnfund.cn,www.hysec.com,www.gyzq.com.cn,www.gf.com.cn,www.cjsc.com
setup_date,20111031,19930525,19970606,19940121,19970724
end_date,-1,-1,-1,-1,-1
main_business,NaN,主要业务:代理买卖证券.,"主营业务:经纪业务,自营投资业务,投行业务,资产管理业务,基金管理业务,期货业务,境外业务国...",主营业务:证券经纪,主营业务:证券经纪


#### 1.2.2 fund_basic 中基金管理人

In [20]:
# 基金数据 
df_fund_basic = pd.read_csv(data_path + fund_basic_name)

In [21]:
df_fund_basic['management'] = df_fund_basic['management'].apply(lambda x : comp_simplify_dict[x] if x in comp_simplify_dict.keys() else x )
df_fund_basic['custodian'] = df_fund_basic['custodian'].apply(lambda x : comp_simplify_dict[x] if x in comp_simplify_dict.keys() else x )

In [22]:
#  df_fund_basic 中 管理人 management
node_fund_management = df_fund_basic.loc[df_fund_basic.management.notnull(),['management']]
node_fund_management = node_fund_management.rename(columns = {'management':'name'})

##### 1.2.3 合并数据

In [23]:
df_fund_company = pd.merge(df_fund_company,node_fund_management,on=['name'],how='outer').drop_duplicates()

In [25]:
df_fund_company[df_fund_company['fullname'].isnull()]

,fullname,name,province,city,address,office,website,setup_date,end_date,main_business,org_code,credit_code
25788,NaN,平安基金,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25952,NaN,东证资管,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26008,NaN,华泰证券资管,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26027,NaN,惠升基金,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26031,NaN,同泰基金,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26039,NaN,中泰资管,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26050,NaN,西藏东财基金,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26054,NaN,朱雀基金,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26058,NaN,淳厚基金,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26062,NaN,博远基金,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


fund_basic 中基金管理人（management）中有 12 个公司 没有 在 资管数据表 fund_company 中出现。 需要额外补全，其全称

In [26]:
# 基金管理人补齐全称
fund_manage_fullname_dict = {
    '东证资管':'东方证券资产管理有限公司',
    '中泰资管':'中泰证券（上海）资产管理有限公司',
    '华泰证券资管':'华华泰证券资产管理公司',
    '华融基金':'华融基金管理有限公司',
    '博远基金':'博远基金管理有限公司',
    '同泰基金':'同泰基金管理有限公司',
    '平安基金':'平安基金管理有限公司',
    '惠升基金':'惠升基金管理有限公司',
    '朱雀基金':'朱雀基金管理有限公司',
    '淳厚基金':'淳厚基金管理有限公司',
    '睿远基金':'睿远基金管理有限公司',
    '西藏东财基金':'西藏东财基金管理有限公司',
    '山西证券':'山西证券股份有限公司',
    '恒丰银行':'恒丰银行股份有限公司',
    '广发银行':'广发银行股份有限公司'
}

In [27]:
df_fund_company['fullname'] = df_fund_company.apply(lambda x:fund_manage_fullname_dict.get(x['name']) if pd.isnull(x['fullname']) else x['fullname'],axis=1 )

In [28]:
df_fund_company['label_manager'] = 'FUND_MANAGER'

In [82]:
df_fund_company.head()

,fullname,name,province,city,address,office,website,setup_date,end_date,main_business,org_code,credit_code,label_manager
0,北京广能投资基金管理有限公司,广能基金,北京,北京市,北京市朝阳区北四环中路27号院5号楼2712-2715A,北京市朝阳区北四环中路27号院5号楼2712-2715A,www.gnfund.cn,20111031.0,-1.0,NaN,584419680,NaN,FUND_MANAGER
1,宏源证券股份有限公司,宏源证券,新疆,乌鲁木齐市,新疆维吾尔自治区乌鲁木齐市文艺路233号宏源大厦,新疆维吾尔自治区乌鲁木齐市文艺路233号宏源大厦,www.hysec.com,19930525.0,-1.0,主要业务:代理买卖证券.,228593068,NaN,FUND_MANAGER
2,国元证券股份有限公司,国元证券,安徽,合肥市,安徽省合肥市梅山路18号,安徽省合肥市梅山路18号,www.gyzq.com.cn,19970606.0,-1.0,"主营业务:经纪业务,自营投资业务,投行业务,资产管理业务,基金管理业务,期货业务,境外业务国...",731686376,91340000731686376P,FUND_MANAGER
3,广发证券股份有限公司,广发证券,广东,广州市,广东省广州市黄埔区中新广州知识城腾飞一街2号618室,"广东省广州市天河区天河北路183-187号大都会广场40楼5楼,7楼,8楼,18楼,19楼,...",www.gf.com.cn,19940121.0,-1.0,主营业务:证券经纪,126335439,91440000126335439C,FUND_MANAGER
4,长江证券股份有限公司,长江证券,湖北,武汉市,湖北省武汉市江汉区新华路特8号,湖北省武汉市江汉区新华路特8号,www.cjsc.com,19970724.0,-1.0,主营业务:证券经纪,700821272,91420000700821272A,FUND_MANAGER


### 1.3 基金托管公司

托管人中存在下面8个没有在大陆上市的公司。

- '中泰证券': 未上市
- '中金公司': HK中金公司(03908)
- '包商银行': 未上市
- '广发银行': 未上市
- '广州农商银行': 广州农商银行 01551.HK
- '徽商银行':  徽商银行03698.HK
- '恒丰银行': 未上市
- '渤海银行': 未上市

In [29]:
node_fund_custodian = df_fund_basic.loc[df_fund_basic.custodian.notnull(),['custodian']].drop_duplicates()
node_fund_custodian = node_fund_custodian.rename(columns = {'custodian':'name'})

In [30]:
node_fund_custodian['label_custodian'] = 'FUND_CUSTODIAN'

In [31]:
node_fund_custodian.shape

(40, 2)

### 1.4 合并数据·

In [32]:
node_companies = pd.merge(df_company_lsit_all,df_fund_company,on=['fullname','name'],how='outer').drop_duplicates()

In [33]:
node_companies = pd.merge(node_companies,node_fund_custodian,on=['name'],how='outer').drop_duplicates()

In [34]:
node_companies[node_companies.fullname.isnull()].name

18971    广发银行
18972    恒丰银行
Name: name, dtype: object

 有**2** 家公司没有拼上,分别是 `恒丰银行`,`广发银行`.
 通过网查,将两个公司的全名补上

In [35]:
node_companies.loc[node_companies.name == '恒丰银行','fullname'] = '恒丰银行股份有限公司'
node_companies.loc[node_companies.name == '广发银行','fullname'] = '广发银行股份有限公司'

In [36]:
node_companies[node_companies.fullname.isnull()]

,ts_code,symbol,name,industry,fullname,market,exchange,list_status,list_date,delist_date,...,address,office,website,setup_date,end_date,main_business,org_code,credit_code,label_manager,label_custodian


In [37]:
print('缺失的资管公司数：',len(set(df_fund_basic.management) - set(node_companies.name)))

set(df_fund_basic.management) - set(node_companies.name)

缺失的资管公司数： 0


set()

### 1.5  处理标签

In [38]:
node_companies['label_listed_company'] = node_companies['label_listed_company'].fillna('')
node_companies['label_manager'] = node_companies['label_manager'].fillna('')
node_companies['label_custodian'] = node_companies['label_custodian'].fillna('')

In [40]:
def join_label(ls):
    return 'COMPANY;'+ ';'.join([c for c in ls  if c !=''])

In [41]:
node_companies['label'] = node_companies.apply(lambda x: join_label([x['label_listed_company'],x['label_manager'],x['label_custodian']]),axis=1)

In [44]:
# node_companies.head().T

In [45]:
# 基于 上市公司信息 company_lsit_all
all_dicts['node_companies'] = {  'fullname':'company:ID(company)', 
                                'name':'short_name', # 股票名称
                                'symbol':'share_code', # 股票全称
                                'market':'market', # 市场类型 （主板/中小板/创业板/科创板）
                                'exchange':'exchange', # 交易所代码
                                'list_status':'list_status', # 上市状态： L上市 D退市 P暂停上市
                                'list_date':'list_date:float', # 上市日期
                                'delist_date':'delist_date:float', # 退市日期
                                'setup_date':'setup_date',   # 成立日期
                                'label':':LABEL'
                           }

In [46]:
types = 'node_companies'
temp = generate_input_files(node_companies,dicts=all_dicts[types] ,types = types,output_path = output_path)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


save header {}..
save_data ...
 ../data/output/node_companies.csv
(18973, 10)


### 节点：行业

In [48]:
all_dicts['node_industries'] = {'industry':'industry:ID(industry)', # 所属行业 
                                'label':':LABEL'
                               }

In [49]:
df_industry = pd.DataFrame({'industry':[c for c in df_company_lsit_all.industry.unique() if not pd.isnull(c)]})
df_industry['label']='INDUSTRY'

In [50]:
df_industry.head()

,industry,label
0,银行,INDUSTRY
1,全国地产,INDUSTRY
2,生物制药,INDUSTRY
3,环境保护,INDUSTRY
4,区域地产,INDUSTRY


In [51]:
types = 'node_industries'
temp = generate_input_files(df_industry,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ../data/output/node_industries.csv
(110, 2)


### 关系：股票-->行业

In [52]:
all_dicts['rel_share_in_industry'] = {'fullname':':START_ID(company)', # 股票代码
                                      'industry':':END_ID(industry)', # 所属行业
                                      'type':':TYPE'
                                      }

In [53]:
rel_share_in_industry = df_company_lsit_all[['industry','fullname']]

rel_share_in_industry = rel_share_in_industry[rel_share_in_industry['fullname'].notnull()&rel_share_in_industry['industry'].notnull()]
rel_share_in_industry['type']='IN_INDUSTRY'

In [54]:
types = 'rel_share_in_industry'
temp = generate_input_files(rel_share_in_industry,dicts=all_dicts[types],types = types, output_path = output_path)

save header {}..
save_data ...
 ../data/output/rel_share_in_industry.csv
(3744, 3)


In [55]:
df_company_lsit_all.head()

,ts_code,symbol,name,industry,fullname,market,exchange,list_status,list_date,delist_date,label_listed_company
0,000001.SZ,000001,平安银行,银行,平安银行股份有限公司,主板,SZSE,L,19910403,-1,LISTED_COMPANY
1,000002.SZ,000002,万科A,全国地产,万科企业股份有限公司,主板,SZSE,L,19910129,-1,LISTED_COMPANY
2,000004.SZ,000004,国农科技,生物制药,深圳中国农大科技股份有限公司,主板,SZSE,L,19910114,-1,LISTED_COMPANY
3,000005.SZ,000005,世纪星源,环境保护,深圳世纪星源股份有限公司,主板,SZSE,L,19901210,-1,LISTED_COMPANY
4,000006.SZ,000006,深振业A,区域地产,深圳市振业(集团)股份有限公司,主板,SZSE,L,19920427,-1,LISTED_COMPANY


### 节点：省份

##### 上市公司基本信息

In [56]:
df_company_detail = pd.read_csv(data_path + company_detail_name)

In [57]:
df_company_detail['symbol'] = df_company_detail['ts_code'].apply(lambda x:x.split('.')[0])

In [58]:
all_dicts['node_province'] = {'province':'province:ID(province)', # 所属行业 
                                'label':':LABEL'
                               }

In [59]:
node_province1 = pd.DataFrame({'province':[c for c in df_company_detail.province.unique() if not pd.isnull(c)]})
node_province2 = pd.DataFrame({'province':[c for c in df_fund_company.province.unique() if not pd.isnull(c)]})
node_province = pd.concat([node_province1,node_province2]).drop_duplicates()
node_province['label']='PROVINCE'

In [60]:
types = 'node_province'
temp = generate_input_files(node_province,dicts=all_dicts[types],types = types, output_path = output_path)

save header {}..
save_data ...
 ../data/output/node_province.csv
(32, 2)


### 节点：城市

In [62]:
all_dicts['node_city'] = {'city':'city:ID(city)', # 所属行业 
                          'label':':LABEL'
                          }

In [63]:
node_city1 = pd.DataFrame({'city':[c for c in df_company_detail.city.unique() if not pd.isnull(c)]})
node_city2 = pd.DataFrame({'city':[c for c in df_fund_company.city.unique() if not pd.isnull(c)]})
node_city = pd.concat([node_city1,node_city2]).drop_duplicates()

node_city['label']='CITY'

In [64]:
types = 'node_city'
temp = generate_input_files(node_city,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ../data/output/node_city.csv
(341, 2)


### 关系：公司-->城市

In [65]:
rel_company_in_city1 = pd.merge(df_company_detail,df_company_lsit_all[['symbol','fullname']],on='symbol',how='left')[['fullname','city']]
rel_company_in_city2 = df_fund_company[['fullname','city']]

rel_company_in_city = pd.concat([rel_company_in_city1,rel_company_in_city2]).drop_duplicates()

rel_company_in_city['type']='IN_CITY'

In [66]:
all_dicts['rel_company_in_city'] = {'fullname':':START_ID(company)', # 股票代码
                                  'city':':END_ID(city)', # 所属城市
                                  'type':':TYPE'
                                  }

In [67]:
rel_company_in_city = rel_company_in_city[rel_company_in_city['city'].notnull()&rel_company_in_city['fullname'].notnull()]


In [68]:
types = 'rel_company_in_city'
temp = generate_input_files(rel_company_in_city,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ../data/output/rel_company_in_city.csv
(18943, 3)


### 关系：城市-->省份

In [71]:
all_dicts['rel_city_in_province'] = {'city':':START_ID(city)', 
                                  'province':':END_ID(province)', 
                                  'type':':TYPE'
                                  }

In [72]:
rel_city_in_province1= df_company_detail[['province','city']]
rel_city_in_province2= df_fund_company[['province','city']]
rel_city_in_province = pd.concat([rel_city_in_province1,rel_city_in_province2]).drop_duplicates()

rel_city_in_province = rel_city_in_province[rel_city_in_province['province'].notnull()&rel_city_in_province['city'].notnull()]
rel_city_in_province['type']='IN_PROVINCE'

In [73]:
types = 'rel_city_in_province'

temp = generate_input_files(rel_city_in_province,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ../data/output/rel_city_in_province.csv
(341, 3)


### 节点：人（上市公司董事高管）

对于公司高管，因为缺少人的唯一身份id，为了避免同名的影响，采用 姓名+性别+生日 进行hash 生成唯一标识。

做法参考：https://github.com/lemonhu/stock-knowledge-graph

In [75]:
import hashlib

In [76]:
def get_md5(string):
    """Get md5 according to the string
    """
    byte_string = string.encode("utf-8")
    md5 = hashlib.md5()
    md5.update(byte_string)
    result = md5.hexdigest()
    return result

In [77]:
df_stk_managers = pd.read_csv(data_path + stk_managers_name)

In [78]:
df_stk_managers['birthday'] = df_stk_managers['birthday'].apply(lambda x: -1 if pd.isnull(x) else int(x))
df_stk_managers['gender'] = df_stk_managers['gender'].apply(lambda x: -1 if x not in ['F','M'] else x)
df_stk_managers['hash_cust'] = df_stk_managers.apply(lambda x:get_md5('{}-{}-{}'.format(x['name'],x['gender'],x['birthday'])),axis=1 )

In [79]:
all_dicts['node_managers'] = {'hash_cust':'manager_id:ID(manager_id)', 
                              'name':'name', 
                              'gender':'gender',
                              'edu':'edu',
                              'national':'national',
                              'birthday':'birthday:long', 
                              'label':':LABEL'
                          }

In [81]:
types = 'node_managers'

df_stk_managers['label'] = 'MANAGER'
node_managers = df_stk_managers[list(all_dicts[types].keys())]

In [82]:
node_managers.head()

,hash_cust,name,gender,edu,national,birthday,label
0,6b549e55dc02acb8483d76625790a70c,徐宏,M,本科,中国,1973,MANAGER
1,7f260c335bc616344b3de75b4ed212f3,张彧,F,硕士,中国,1972,MANAGER
2,126b1cb357ab8b4b777708c91ec930f5,张近东,M,本科,中国,1963,MANAGER
3,c1576573eef4f6061b59f1c2491d56f6,肖忠祥,M,本科,中国,1971,MANAGER
4,06a551a6cf1e98790df9f1dbb1419841,汪晓玲,F,本科,中国,1973,MANAGER


In [83]:
node_managers = node_managers.sort_values(['name','gender','edu','national']).drop_duplicates('hash_cust',keep='first')

temp = generate_input_files(node_managers,dicts=all_dicts[types],types = types, output_path = output_path)

save header {}..
save_data ...
 ../data/output/node_managers.csv
(163613, 7)


### 关系：公司（上市公司）--> 人(董事高管)

计划将高管关系分为六类

|中文| 英文|数量|占比|
|--|--|--|--|
|董事会成员 |Has_Director|125040|0.278736|
|监事|Has_Supervisor|60332|0.134491|
|委员会成员 |Has_Committee|41544|0.092609|
|高管 | Has_Manager |102271|0.227980|
|核心技术人员 |Has_Core_Technical_Staff|20|0.000045|
|其他 | Has_Othoer_Manager|119389|0.266139|


In [84]:
all_dicts['rel_listed_company_has_manager'] = {'fullname':':START_ID(company)', 
                                              'hash_cust':':END_ID(manager_id)', 
                                              'begin_date':'begin_date:long',
                                              'end_date':'end_date:long',
                                              'title':'title',
                                              'lev':':TYPE'
                                              }

In [85]:
df_stk_managers['symbol'] = df_stk_managers['ts_code'].apply(lambda x:x.split('.')[0])

df_stk_managers['begin_date'] = df_stk_managers['begin_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_stk_managers['end_date'] = df_stk_managers['end_date'].apply(lambda x: -1 if pd.isnull(x) else int(x))
df_stk_managers['ann_date'] = df_stk_managers['ann_date'].apply(lambda x: -1 if pd.isnull(x) else int(x))

In [86]:
rel_listed_company_has_manager = pd.merge(df_stk_managers,df_company_lsit_all[['symbol','fullname']],on='symbol',how='left')

In [69]:
# 部分为空的的人员，为退市的人员
# temp = rel_listed_company_has_manager[rel_listed_company_has_manager['fullname'].isnull()]

In [87]:
types = 'rel_listed_company_has_manager'

rel_listed_company_has_manager = rel_listed_company_has_manager[rel_listed_company_has_manager['fullname'].notnull()&rel_listed_company_has_manager['hash_cust'].notnull()]

In [88]:
rel_listed_company_has_manager = rel_listed_company_has_manager.sort_values(['fullname','hash_cust','title','begin_date','end_date'],ascending=False).drop_duplicates(['fullname','hash_cust','title','begin_date'],keep='first')

In [89]:
rel_listed_company_has_manager.head()

,ts_code,ann_date,name,gender,lev,title,edu,national,birthday,begin_date,end_date,hash_cust,label,symbol,fullname
154169,002601.SZ,20170425,许晓斌,M,其他,薪酬与考核委员会委员,硕士,中国,197701,20140404,-1,f52079b3b6db2cc1508dbc9caa2d678c,MANAGER,002601,龙蟒佰利联集团股份有限公司
154178,002601.SZ,20170425,许晓斌,M,其他,薪酬与考核委员会召集人,硕士,中国,197701,20140404,-1,f52079b3b6db2cc1508dbc9caa2d678c,MANAGER,002601,龙蟒佰利联集团股份有限公司
154198,002601.SZ,20170425,许晓斌,M,董事会成员,独立董事,硕士,中国,197701,20140404,-1,f52079b3b6db2cc1508dbc9caa2d678c,MANAGER,002601,龙蟒佰利联集团股份有限公司
154209,002601.SZ,20140408,许晓斌,M,其他,战略委员会委员,硕士,中国,197701,20140404,20170421,f52079b3b6db2cc1508dbc9caa2d678c,MANAGER,002601,龙蟒佰利联集团股份有限公司
154160,002601.SZ,20170425,许晓斌,M,其他,审计委员会委员,硕士,中国,197701,20140404,-1,f52079b3b6db2cc1508dbc9caa2d678c,MANAGER,002601,龙蟒佰利联集团股份有限公司


In [90]:
temp = generate_input_files(rel_listed_company_has_manager,dicts=all_dicts[types],types = types,output_path=output_path)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


save header {}..
save_data ...
 ../data/output/rel_listed_company_has_manager.csv
(432429, 6)


### 节点：基金

In [91]:
df_fund_basic['fund'] = df_fund_basic['ts_code'].apply(lambda x: x.split('.')[0])
df_fund_basic['market'] = df_fund_basic['ts_code'].apply(lambda x: x.split('.')[1])

In [92]:
df_fund_basic['found_date'] = df_fund_basic['found_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_basic['due_date'] = df_fund_basic['due_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_basic['list_date'] = df_fund_basic['list_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_basic['issue_date'] = df_fund_basic['issue_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_basic['delist_date'] = df_fund_basic['delist_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )

In [94]:
df_fund_basic['issue_amount'] = df_fund_basic['issue_amount'].apply(lambda x:float(x))
df_fund_basic['m_fee'] = df_fund_basic['m_fee'].apply(lambda x:float(x))
df_fund_basic['c_fee'] = df_fund_basic['c_fee'].apply(lambda x:float(x))
df_fund_basic['min_amount'] = df_fund_basic['min_amount'].apply(lambda x:float(x))

df_fund_basic['duration_year'] = df_fund_basic['duration_year'].apply(lambda x:float(x))

In [96]:
all_dicts['node_funds'] =  {'fund':'fund_code:ID(fund_code)', 
                            'name':'name', 
                            'fund_type':'fund_type', # 投资类型
                            'invest_type':'invest_type', # 投资风格
                            'type':'type',  # 基金类型 
                            'benchmark':'benchmark', # 业绩比较基准
                            'market':'market', #  场外'OF', 场内 'SH', 'SZ'
                            'found_date':'found_date:long',  # 成立日期
                            'delist_date':'delist_date:long',  # 退市日期
                            'status':'status',  #  存续状态: D摘牌 I发行 L已上市
                            'label':':LABEL' }

#### 数据预处理

对于 这几个时间，有两种处理方式：1.作为节点属性；2.作为关系属性

- 'found_date'  # 成立日期
- 'due_date'    # 到期日期
- 'list_date'  # 上市时间
- 'issue_date'  # 发行日期
- 'delist_date'   # 退市日期

每个基金都有唯一的5个时间.
但是 因为 Neo4j 不持支对于边添加索引.所以考虑查询效率,将这些属性放于节点上.

In [98]:
types = 'node_funds'

df_fund_basic['label'] = 'FUND'
node_funds = df_fund_basic[list(all_dicts[types].keys())]

In [99]:
node_funds = node_funds.sort_values(['fund','found_date','delist_date'],ascending=False).drop_duplicates('fund',keep='first')

In [100]:
node_funds.head()

,fund,name,fund_type,invest_type,type,benchmark,market,found_date,delist_date,status,label
6984,F450005,国富潜力组合H港币,混合型,混合型,契约型开放式,MSCI中国A股指数*85%+中债国债总指数(全价)*10%+同业存款息率*5%,OF,20160229,-1,L,FUND
6985,F450004,国富潜力组合H美元,混合型,混合型,契约型开放式,MSCI中国A股指数*85%+中债国债总指数(全价)*10%+同业存款息率*5%,OF,20160229,-1,L,FUND
6934,F217003,招商安泰债券H,债券型,债券型,契约型开放式,中证国债指数收益率*95%+同业存款利率*5%,OF,20160317,-1,L,FUND
6532,F202108,南方润元纯债H,债券型,债券型,契约型开放式,中证全债指数,OF,20160713,-1,L,FUND
6282,F202017,南方深成ETF联接H,股票型,被动指数型,契约型开放式,深证成份指数收益率*95%+银行活期存款利率(税后)*5%,OF,20160830,-1,L,FUND


In [101]:
temp = generate_input_files(node_funds,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ../data/output/node_funds.csv
(10859, 11)


### 关系：基金-->托管人

In [102]:
temp = node_companies[['name','fullname']].rename(columns = {'name':'custodian'})

In [103]:
rel_fund_has_custodian = pd.merge(df_fund_basic,temp, on='custodian',how='left')[['fund','fullname']]

In [104]:
all_dicts['rel_fund_has_custodian'] = {'fund':':START_ID(fund_code)',   # TS基金代码
                                      'fullname':':END_ID(company)',   # 股票代码
                                      'type':':TYPE'
                                     }

In [105]:
rel_fund_has_custodian['type']='HAS_CUSTODIAN'

In [106]:
types = 'rel_fund_has_custodian'

temp = generate_input_files(rel_fund_has_custodian,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ../data/output/rel_fund_has_custodian.csv
(10923, 3)


In [107]:
rel_fund_has_custodian.head().T

,0,1,2,3,4
fund,515070,515330,515310,501089,515080
fullname,中国银行股份有限公司,招商证券股份有限公司,招商银行股份有限公司,中国民生银行股份有限公司,上海浦东发展银行股份有限公司
type,HAS_CUSTODIAN,HAS_CUSTODIAN,HAS_CUSTODIAN,HAS_CUSTODIAN,HAS_CUSTODIAN


### 关系：基金-->管理人

In [108]:
temp = node_companies[['name','fullname']].rename(columns = {'name':'management'})

In [109]:
rel_fund_has_management = pd.merge(df_fund_basic,temp, on='management',how='left')[['fund','fullname']]

In [110]:
all_dicts['rel_fund_has_management'] = {'fund':':START_ID(fund_code)',   # TS基金代码
                                      'fullname':':END_ID(company)',   # 股票代码
                                      'type':':TYPE'
                                     }

In [111]:
rel_fund_has_management['type']='HAS_MANAGEMENT'

In [112]:
types = 'rel_fund_has_management'

temp = generate_input_files(rel_fund_has_management,dicts=all_dicts[types],types = types,output_path = output_path)

save header {}..
save_data ...
 ../data/output/rel_fund_has_management.csv
(10935, 3)


In [113]:
rel_fund_has_management.head().T

,0,1,2,3,4
fund,515070,515330,515310,501089,515080
fullname,华夏基金管理有限公司,天弘基金管理有限公司,汇添富基金管理股份有限公司,方正富邦基金管理有限公司,招商基金管理有限公司
type,HAS_MANAGEMENT,HAS_MANAGEMENT,HAS_MANAGEMENT,HAS_MANAGEMENT,HAS_MANAGEMENT


### 关系: 公募基金持仓数据 

In [114]:
df_fund_portfolio = pd.read_csv(data_path + fund_portfolio_name )

In [115]:
df_fund_portfolio['fund'] = df_fund_portfolio['ts_code'].apply(lambda x: x.split('.')[0])

In [116]:
df_fund_portfolio['symbol'] = df_fund_portfolio['symbol'].apply(lambda x: x.split('.')[0])

In [117]:
all_dicts['rel_fund_listed_company_portfolio'] = {'fund':':START_ID(fund_code)',   # TS基金代码
                                                  'fullname':':END_ID(company)',   # 股票代码
                                                  'ann_date':'ann_date:float',   # 公告日期
                                                  'end_date':'end_date:float' ,  # 截止日期
                                                  'mkv':'mkv:float' ,  # 持有股票市值(元)
                                                  'amount':'amount:float' ,  # 持有股票数量（股）
                                                  'stk_mkv_ratio':'stk_mkv_ratio:float',   # 占股票市值比
                                                  'stk_float_ratio':'stk_float_ratio:float',   # 占流通股本比例
                                                  'type':':TYPE'
                                                 }

In [118]:
df_fund_portfolio['ann_date'] = df_fund_portfolio['ann_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )
df_fund_portfolio['end_date'] = df_fund_portfolio['end_date'].apply(lambda x: -1 if pd.isnull(x) else int(x) )

df_fund_portfolio['amount'] = df_fund_portfolio['amount'].apply(float)
df_fund_portfolio['stk_mkv_ratio'] = df_fund_portfolio['stk_mkv_ratio'].apply(float)
df_fund_portfolio['stk_float_ratio'] = df_fund_portfolio['stk_float_ratio'].apply(float)
df_fund_portfolio['mkv'] = df_fund_portfolio['mkv'].apply(float)

In [119]:
rel_fund_listed_company_portfolio = pd.merge(df_fund_portfolio,df_company_lsit_all[['symbol','fullname']],on='symbol',how='left')

In [120]:
rel_fund_listed_company_portfolio = rel_fund_listed_company_portfolio.sort_values(['fund','fullname','ann_date','end_date'],ascending=False).drop_duplicates(['fund','fullname'],keep='first')

In [121]:
rel_fund_listed_company_portfolio['type']='IN_PORTFOLIO'

In [124]:
rel_fund_listed_company_portfolio.shape

(1449174, 11)

In [123]:
types = 'rel_fund_listed_company_portfolio'

temp = generate_input_files(rel_fund_listed_company_portfolio,dicts=all_dicts[types],types = types,output_path = output_path)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


save header {}..
save_data ...
 ../data/output/rel_fund_listed_company_portfolio.csv
(1449174, 9)
